# In this notebook, I log metrics of Neural Prophet model in which I try adding and removing weekly seasonality for each store product pair

In [ ]:
%cd ..

In [ ]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import mlflow
import pickle
import sklearn.ensemble
from sklearn.datasets import load_diabetes
from neuralprophet import NeuralProphet
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error

## Prepare Data fror training

In [ ]:
df_train = pd.read_csv('./data/train.csv', low_memory=False)
df_test = pd.read_csv('./data/test.csv', low_memory=False)

# Transform Date column into datetime format
df_train['ds'] = pd.to_datetime(df_train['Date'])
df_test['ds'] = pd.to_datetime(df_test['Date'])

df_train.rename(columns={'number_sold':'y'}, inplace=True)
df_test.rename(columns={'number_sold':'y'}, inplace=True)
df_train['ID'] = df_train['store'].astype('str') + '_' + df_train['product'].astype('str')
df_test['ID'] = df_test['store'].astype('str') + '_' + df_test['product'].astype('str')

In [ ]:
df = df_train.drop_duplicates(subset=['ID', 'ds'])
df['ds'] = df['ds'].dt.date

## Train and log results

In [ ]:
mlflow.set_experiment("neuralprophet_forecasting")
with mlflow.start_run(run_name="local_weekly_seasonality"):
    mlflow.log_param("trend_global_local", 'local')
    mlflow.log_param("season_global_local", 'local')
    mlflow.log_param("weekly_seasonality_glocal_mode", 'local')
    mlflow.log_param("daily_seasonality",False)
    mlflow.log_param("n_lags", 375)
    mlflow.log_param("n_forecasts", 365)
    mlflow.log_param("epochs",20)
    

    # Log a model

    model = NeuralProphet(
        trend_global_local = 'local',
        season_global_local= 'local',
        daily_seasonality=False,
        weekly_seasonality_glocal_mode='local',
        n_lags= 375,
        n_forecasts=365,
        epochs=20
    )

    metrics = model.fit(df[['ID', 'ds', 'y']], freq='D')
    df_pred = model.predict(model.make_future_dataframe(df_train[['ds', 'ID', 'y']]))
    cols = [x for x in df_pred.columns if 'hat' in x]
    df_pred['y_pred'] = df_pred[cols].sum(axis=1)

    df_comb = df_pred.merge(
        df_test, 
        on=['ds', 'ID'], 
        suffixes=['_preds', '_actual']
        )
    
    mape = mean_absolute_percentage_error(df_comb['y_actual'], df_comb['y_pred'])
    with open("neural_prophet_model.pkl", "wb") as f:
        pickle.dump(model, f)

    mlflow.log_artifact("neural_prophet_model.pkl")
    mlflow.log_metric("mape", mape)
    
